In [180]:
import pandas as pd
import re
import emoji
from ftfy import fix_text 
import unidecode
from langdetect import detect, detect_langs
import spacy

In [181]:
# df = pd.read_json(r'C:\Users\rockm\golden-globe-337\gg2013.json')['text']
df = pd.read_json('gg2013.json')['text']

In [182]:
df.to_csv('text.csv')

In [183]:
df

0                    JLo's dress! #eredcarpet #GoldenGlobes
1         What's making Sofia Vergara's boobs stay like ...
2         RT @FabSugar: Kerry Washington is EVERYTHING. ...
3            Anne Hathaway has got me living. #GoldenGlobes
4         Jennifer Lopez's lace dress? Thoughts? #Golden...
                                ...                        
174638    RT @authorViviAnna: I was sad that Mandy Patin...
174639    RT @_ItzelMartinez_: Jennifer Lawrence aceptan...
174640    Golden Globes, lots of fashion messes...but gl...
174641    Did they have mug shots at the golden globes?!...
174642    Says @BenAffleck: "I also didn't get the Actin...
Name: text, Length: 174643, dtype: object

In [184]:
def clean(text):
    text = fix_text(text)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|pic.twitter\S+', '', text)
    # Remove emojis
    # new_text = unidecode(text)
    text = emoji.replace_emoji(text, replace='')
    # Remove hashtags
    text = re.sub(r'#\S+', '', text)
    
    text = re.sub(r'[^A-Za-z0-9@ "]+', '', text)
    
    text = re.sub(' +', ' ', text)
    
    text = text.lower()
    
    # if text.str.strip() != '':
    #     detect(text.split(' ')[0])
    
    return text

In [185]:
df = df.apply(clean)

In [186]:
cleaned_data = df[df.str.strip() != ""]

In [187]:
cleaned_data.to_csv('text_cleaned.csv')

In [188]:
cleaned_data = pd.read_csv('text_cleaned.csv')['text']

In [189]:
cleaned_data[cleaned_data.apply(lambda x: re.search('(?=.*award|AWARD)(?=.*wins|Wins|WINS|winner|WINNER).*', x) != None)]

1462      rt @possessionista if jennifer lawrence starts...
2944      id watch the but scripted tv wins out over awa...
6441      golden globes 2013 list of winners from this y...
6932                      bill murray wins the beard award 
8152      the begins hope django unchained wins an award...
                                ...                        
173191    rt @instyle looking every bit a winner stopped...
173233    golden globes 2013 british stars take home awa...
173291    golden globes 2013 british stars take home awa...
173330    rt @thr complete list of the winners at the 70...
173387    winner at our @goldenglobes party practice rou...
Name: text, Length: 646, dtype: object

In [226]:
# win_data = cleaned_data[cleaned_data.apply(lambda x: re.search(r'\b(win|wins|won|winner|victor|victory|receive|received|receives|awarded)\b', x) != None)]
win_data = cleaned_data[cleaned_data.apply(lambda x: re.search(r'\bwins\b', x) != None)]

In [191]:
win_data

In [230]:
win_data.to_csv("wins.csv")

In [193]:
from spacy import displacy

### New Idea

1) loop through cleaned dataframe to find tweets with "win," "present," "nominate," "best" (maybe), "award" (maybe)
2) with a match, get any names

In [194]:
test_tweet = "rt @galaxiemag lifetime achievement award winner jodie foster is 50 she has been acting for 47 of those years amazing"

In [195]:
test_tweet = "christoph waltz wins best supporting actor for django unchained"

In [196]:
alleged_winner = ""
subject = []
actual_winner = ""

nlp = spacy.load('en_core_web_sm')

doc = nlp(test_tweet)

def extract_entities_2(doc):
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

for ent in extract_entities_2(doc):
    if ent[1] == "PERSON":
        alleged_winner = ent[0]

# for token in doc:
#     if (token.dep_ == 'nsubj'):
#         nsubj = token.text
#     elif (token.dep_ == 'dobj'):
#         dobj = token.text

for token in doc:
    # extract subject
    if (token.dep_ == 'nsubj'):
        subject.append(token.text)

In [197]:
def pattern(word, text): 
    pat = re.compile(re.escape(word), re.IGNORECASE) 
    return bool(re.search(pat, text))

In [198]:
entities = extract_entities_2(doc)
for s in subject:
    for e, _ in entities:
        if pattern(s, e):
            print(e)

In [199]:
doc.ents

In [235]:
nlp = spacy.load('en_core_web_sm')
def get_winner(text):
    alleged_winner = ""
    subject = []
    actual_winner = ""
    
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    
    # return entities
    
    # for ent in entities:
    #     if ent[1] == "PERSON":
    #         alleged_winner = ent[0]
            
    for token in doc:
        # extract subject
        if (token.dep_ == 'nsubj'):
            subject.append(token.text)
    
    entities = extract_entities_2(doc)
    for s in subject:
        for e, _ in entities:
            if pattern(s, e):
                return e

In [201]:
len(win_data)

In [202]:
spacy_size = 1500

In [203]:
test = win_data.head(spacy_size).to_frame()

In [204]:
test['spacy'] = win_data.head(spacy_size).apply(get_winner)

In [205]:
test.to_csv('spacy_info.csv')

In [206]:
text = "golden globe winner damian lewis wins best actor in a tv drama for ""homeland "
doc = nlp(text)
entities = [(ent.text, ent.label_) for ent in doc.ents]
subject = []

# for ent in entities:
#     if ent[1] == "PERSON":
#         alleged_winner = ent[0]
        
for token in doc:
    # extract subject
    # if (token.dep_ == 'nsubj'):
    print(token, token.dep_)

subject 

In [207]:
nom_data = cleaned_data[cleaned_data.apply(lambda x: re.search('(nominee|nominate|nominated|nominees)', x) != None)]

In [208]:
nom_data

49        rt @lokipage times we wish good luck tonight t...
61        rt @twittermovies nominee @officialadele on th...
154       ive seen a grand total of one nominated film s...
338       rt @zap2itrick can kerry washington be nominat...
505       rt @twittermovies nominee @officialadele on th...
                                ...                        
173059    the were amazing im so happy argo won but i th...
173107    rt @oskrnyc "7 cajas" should have been nominat...
173130    "@itsjerrah i should be nominated as best acto...
173172    congrats to all the 2013 golden globes winners...
173203    rt @mapeel yes should have been nominated for ...
Name: text, Length: 1385, dtype: object

In [209]:
from project import *

In [210]:
a = "christoph waltz wins best supporting actor for django unchained"

In [217]:
cleaned_data.head(50).apply(lambda x: extract_award_details(x))

0     {'winner': None, 'nominees': None, 'presenters...
1     {'winner': None, 'nominees': None, 'presenters...
2     {'winner': None, 'nominees': None, 'presenters...
3     {'winner': None, 'nominees': None, 'presenters...
4     {'winner': None, 'nominees': None, 'presenters...
5     {'winner': None, 'nominees': None, 'presenters...
6     {'winner': None, 'nominees': None, 'presenters...
7     {'winner': None, 'nominees': None, 'presenters...
8     {'winner': None, 'nominees': None, 'presenters...
9     {'winner': None, 'nominees': None, 'presenters...
10    {'winner': None, 'nominees': None, 'presenters...
11    {'winner': None, 'nominees': None, 'presenters...
12    {'winner': None, 'nominees': None, 'presenters...
13    {'winner': None, 'nominees': None, 'presenters...
14    {'winner': None, 'nominees': None, 'presenters...
15    {'winner': None, 'nominees': None, 'presenters...
16    {'winner': None, 'nominees': None, 'presenters...
17    {'winner': None, 'nominees': None, 'presen

In [223]:
from capture_group import *

In [228]:
find_award_winner(a)

'The award winner is: christoph waltz'

In [234]:
win_data.head(50).apply(lambda x: find_award_winner(x))

349                          No award mention found.
607                          No award mention found.
619               Possible award winner: rt @morocca
836                          No award mention found.
1462        Possible award winner: jennifer lawrence
1466                         No award mention found.
1783              Possible award winner: rt @morocca
1876              Possible award winner: rt @morocca
1919            Possible award winner: anne hathaway
2185                         No award mention found.
2363                         No award mention found.
2507                         No award mention found.
2826                         No award mention found.
2944                         No award mention found.
3088      Possible award winner: rt @vampirediariecw
3168              Possible award winner: rt @morocca
3320                  Possible award winner: kathryn
4191                         No award mention found.
4282                         No award mention 